In [ ]:
!git clone https://github.com/SKplanet/Dialog-KoELECTRA.git

Cloning into 'Dialog-KoELECTRA'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 196 (delta 15), reused 6 (delta 6), pack-reused 174
Receiving objects: 100% (196/196), 42.95 MiB | 12.55 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [ ]:
!pip install transformers

tokeninzer 수정 위해 기존 KoElectra Vocab에 단어 추가 - '파이썬'만 추가


In [ ]:
pip install autocorrect # 오타 교정

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=f8db06c19e853382d4cf775776b32fb58a8fb297ac0e65a2635b6da131945da8
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect


In [ ]:
pip install googletrans==4.0.0-rc1 # 언어 통일(papagoAPI 대용)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=ace5d06aebb38843a007d4050239015140473f7ffd31dc8644372862f1179503
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.0.1
    Uninsta

In [ ]:
from autocorrect import Speller # 오타 교정
# maek => make
import requests
import googletrans # 번역

def fix_typo(sentence, language): # 오타 교정
  spell = Speller(lang=language)

  # split the sentence into words
  words = sentence.split()

  # find and fix typo
  correct_words = [spell(word)+' ' for word in words]

  # make a sentence
  correct_sentence = ''.join(correct_words)
  return correct_sentence


# 타 언어 제거, 번역
# using 파이썬 modify this code => using python modify this code

def mono_lang(sentence, language): # input sentence, 'en', 'ko' ...etc
  # general translation
  source_lang = 'ko'
  target_lang = language
  text = sentence

  # papago API
  client_id = 'EMmKltfYKVWFG61jU5OW'
  client_secret = 'i5npjLvENG'

  # API 엔드포인트 및 헤더 설정
  url = 'https://openapi.naver.com/v1/papago/n2mt'
  headers = {
      'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
      'X-Naver-Client-Id': client_id,
      'X-Naver-Client-Secret': client_secret,
  }

  # en, ko, 번역 텍스트
  data = {
      'source': source_lang,
      'target': target_lang,
      'text': text,
  }

  # API 요청
  response = requests.post(url, headers=headers, data=data)

  # JSON 응답 파싱
  result = response.json()

  # 번역된 텍스트 반환
  translated_text = result['message']['result']['translatedText']
  print('1차 번역 :',translated_text) # 한->영

  data = {
      'source': target_lang,
      'target': source_lang,
      'text': translated_text,
  }

  # API 요청2
  response = requests.post(url, headers=headers, data=data)

  # JSON 응답 파싱2
  result = response.json()

  translated_text = result['message']['result']['translatedText']
  print('2차 번역 :',translated_text) # 영->한

  return translated_text


In [ ]:
import torch

def detokenize(tokens): # 띄어쓰기 제공
    words = []
    current_word = ""
    for token in tokens:
        if token.startswith("##"):
            current_word += token[2:]
        else:
            if current_word:
                words.append(current_word)
            current_word = token

    if current_word:
        words.append(current_word)

    return " ".join(words)


def sentence_split_generate(input_text):
  result = tokenizer.tokenize(input_text)
  print('result : ', result)
  # ['', '[CLS]', '1', '##에', '##서', '10', '##까', '##지', '더', '##하', '##고', ',', '그', '값', '##을', '10', '##으로', '나누', '##는', '파이', '##썬', '코드', '##를', '만들', '##어', '##줘', '.']


# 문장의 목적을 따로 분류하도록 모델을 fine-tuning시켜 강조?


  return_sentence = []
  current_sent = ""

  remove_point = ['[CLS]', '[SEP]']

  for word in result:

    if word in remove_point: # 앞뒤 CLS, SEP 삭제
      continue

  final_result = detokenize(result) # 띄어쓰기 포함해서 문장 합치기

  # if word.endswith('.') or word.endswith(','): # .나 ,로 문장에 줄바꿈
  #   return_sentence.append(current_sent.strip())
  #   current_sent = ""
  # print('return : ', return_sentence)
  return final_result

def generate_custom_sentence(prompt, max_length=50, temperature=1.0): # dialog-koelectra이용해 문장 이쁘게 다듬기(최종)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=max_length, temperature=temperature, num_beams=5)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text


In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch


# chatGPT 프롬프트가 대화형이므로 한국어 대화형에 최적화된 모델인 skplanet - dialog-koElectra를 사용
tokenizer = ElectraTokenizer.from_pretrained("skplanet/dialog-koelectra-small-discriminator")
model = ElectraForSequenceClassification.from_pretrained("skplanet/dialog-koelectra-small-discriminator")


tokenizer_config.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/319k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/58.9M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at skplanet/dialog-koelectra-small-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## main ##

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# 랜덤 시드
set_seed(42)

#update tokenizer process
add_token_num = tokenizer.add_tokens(['파이썬'])# add token in vocab

# update tokenizer
model.resize_token_embeddings(len(tokenizer))

# Sentence to be paraphrased => 여기를 input값으로 수정하면 됨!!
input_sentence = "namne을 질문해, 그리고 버튼을 누르면 이름이랑 랜덤 번호가 출력되도록 파이썬코드짜줘." # => make a python code which adds from 1 to 10 (이정도까지 오타가 나진 않을거같긴함)
print("입력 : ", input_sentence)

# Tokenize the sentence / preprocessing
language = 'en' # target language
fixed_typo_sentence = fix_typo(input_sentence, language) # 특수문자, 오타, 타 언어, 띄어쓰기 교정
print("오타 교정 : ",fixed_typo_sentence)
fixed_language = mono_lang(fixed_typo_sentence, language) # 타 언어 제거 - (input언어가 한국어인 경우만 상정) 파파고 api 이용->번역 2회
print('언어 통일 : ', fixed_language)

# conjunctions = ['그리고', '또한']  # 문장 분할시 추가 접속사들

fixed_lang = sentence_split_generate(fixed_language) # 문장 분할해서 새 문장 생성(줄바꿈)
print("문장 분할 : ",fixed_lang)


# 추가 문맥제공, 강조, trigger sentence 추가 / 문맥, 강조 후에 추가
trigger_sent = "\n이 과정을 순서대로 알려줘. 코드는 한번에 전체 코드를 이어서 보여줘." # 변형 가능 // process in order / print in order . . .

# gui를 사용해 맥락 제공 -> 맥락에 맞는 버튼을 눌러서 트리거 문장 추가
triggerSentences = ['코드를 한 번에 보여줘.',
                    '맞춤법이 틀린 부분을 수정해줘.',
                    '모든 코드에 주석을 달아줘.'] # 추가될 수 있음

final_output = fixed_lang + trigger_sent

# 문장 분할된 값을 모델에 넣어서 문장 이쁘게 다듬기
print("\nfinal output using dialog-koElectra : ", final_output)


입력 :  namne을 질문해, 그리고 버튼을 누르면 이름이랑 랜덤 번호가 출력되도록 파이썬코드짜줘.
오타 교정 :  name을 질문해, 그리고 버튼을 누르면 이름이랑 랜덤 번호가 출력되도록 파이썬코드짜줘. 
1차 번역 : Ask the name, and set the Python code so that when you press the button, the name and random number are displayed.
2차 번역 : 이름을 묻고, 버튼을 누르면 이름과 난수가 표시되도록 파이썬 코드를 설정한다.
언어 통일 :  이름을 묻고, 버튼을 누르면 이름과 난수가 표시되도록 파이썬 코드를 설정한다.
result :  ['이름', '##을', '묻', '##고', ',', '버튼', '##을', '누르', '##면', '이름', '##과', '난', '##수가', '표시', '##되', '##도록', '파이썬', '코드', '##를', '설정', '##한다', '.']
문장 분할 :  이름을 묻고 , 버튼을 누르면 이름과 난수가 표시되도록 파이썬 코드를 설정한다 .

final output using dialog-koElectra :  이름을 묻고 , 버튼을 누르면 이름과 난수가 표시되도록 파이썬 코드를 설정한다 .
이 과정을 순서대로 알려줘. 코드는 한번에 전체 코드를 이어서 보여줘.


@misc{DialogKoELECTRA,
  author       = {Wonchul Kim and Junseok Kim and Okkyun Jeong},
  title        = {Dialog-KoELECTRA: Korean conversational language model based on ELECTRA model},
  howpublished = {\url{https://github.com/skplanet/Dialog-KoELECTRA}},
  year         = {2021},
}